In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# %load_ext autotime

In [ ]:
# Load all data from file
with open('../input/hashcode-drone-delivery/busy_day.in','r') as fd:
    # Load base params
    rows, columns,drones,turns,max_payload = [ int(x) for x in fd.readline().split(' ')]
    product_types_count = int(fd.readline())

    # Products weights
    arr_products_weights = np.empty(product_types_count, dtype = "int32")
    weights = fd.readline().split(' ')
    for i in range(0,product_types_count):
        arr_products_weights[i] = int(weights[i])

    # stock
    warehouses_count = int(fd.readline())
    arr_warehouses_locations = np.empty((warehouses_count,2), dtype="int32")
    arr_inventories = np.empty((product_types_count,warehouses_count), dtype = 'int32')
    for i in range(warehouses_count):
        arr_warehouses_locations[i,:] = [ int(x) for x in fd.readline().split(' ')]
        arr_inventories[:,i] = [ int(x) for x in fd.readline().split(' ')]
    df_warehouses_locations = pd.DataFrame(arr_warehouses_locations,columns=['row','column'])
    df_inventories = pd.DataFrame(arr_inventories, columns = [x for x in range(warehouses_count)])

    # orders
    orders_count = int(fd.readline())
    arr_orders = np.empty((orders_count,3),'int32')
    df_orders_inventory = pd.DataFrame(columns=["order_id","product_id","product_count"], dtype='int32')
    for i in range(orders_count):
        # orders table
        row,column = [int(x) for x in fd.readline().split(' ')]
        items_count = int(fd.readline())
        arr_orders[i,:] = row,column,items_count
        # orders items table
        order_items = [ int(x) for x in fd.readline().split(' ')]
        product_ids, product_counts = np.unique(order_items,return_counts = True)
        arr_order_items = np.empty((len(product_ids),3),'int32')
        arr_order_items[:,0] = i # set order id for all rows
        arr_order_items[:,1] = product_ids
        arr_order_items[:,2] = product_counts
        df_order_items=pd.DataFrame(arr_order_items, columns = ["order_id","product_id","product_count"])
        df_orders_inventory = df_orders_inventory.append(df_order_items,ignore_index=True)
    df_orders = pd.DataFrame(arr_orders, columns = ["row","column","items_count"])

In [ ]:
df_orders.head()

In [ ]:
# Helper table  - distances between all orders
df_orders_dist = pd.DataFrame(columns = df_orders.index,index = df_orders.index)
for order_no in df_orders.index:
    r1 = df_orders["row"].loc[order_no]
    c1 = df_orders["column"].loc[order_no]
    r2 = df_orders["row"]
    c2 = df_orders["column"]
    arr_dists = ((r1 - r2)**2 + (c1 - c2)**2  )**0.5
    df_orders_dist.loc[order_no] = arr_dists
df_orders_dist

In [ ]:
# convenient unit of weight - a share of the drone's maximum load
# calculate the weights of all orders in these units
arr_orders_weight = np.empty(orders_count,'float64')
for order_id in range(orders_count):
    order_items = df_orders_inventory[df_orders_inventory['order_id'] == order_id]
    order_weight = np.sum(arr_products_weights[order_items['product_id']] * \
                    order_items['product_count'])/ \
                  float(max_payload)
    arr_orders_weight[order_id] = order_weight
df_orders['weight'] = arr_orders_weight

In [ ]:
# Let's estimate - for how many drone flights can orders be delivered in current data?
df_orders['weight'].hist(bins=50)

# Let's first look at the data

In [ ]:
# Render the map:
# how are warehouses and orders distributed on the map?
plt.scatter(df_orders['row'],df_orders['column'],c='blue',marker=".",label='orders')
plt.scatter(arr_warehouses_locations[:,0],arr_warehouses_locations[:,1],c='red',label='warehouses')
plt.legend()
plt.grid()
fig = plt.gcf()
fig.set_size_inches(15, 10)


In [ ]:
# Which warehouses are the most loaded in terms of the number of goods in pieces?
print(df_inventories.sum(axis=0))

# General approach to solution

1. We will reserve all goods in warehouses for specific orders, we will receive a table with reserves
2. We will calcalute drone routes that will contain specific deliveries of reserves
3. Distribute routes between existing drones
4. Write a program for the drones and prepare the submission


### Reservation procedure
We move based on warehouses and the goods available on them.
For each product in the warehouse - we are looking for orders closest to the warehouse with this product and form a reserve


In [ ]:
# we calculate for each order and add the distance to each of the warehouses 
# save results in the orders table so that later you can quickly sort by them
# add columns like wh_0, wh_1 and so on

for wh in df_warehouses_locations.index:
    df_orders['wh_'+str(wh)] = ((df_orders['row'] - df_warehouses_locations['row'][wh]) ** 2 + (df_orders['column'] - df_warehouses_locations['column'][wh]) ** 2)**0.5
# total distance from each warehouse to all orders we have
df_orders[["wh_%d" % wh_id for wh_id in range(warehouses_count)]].sum(axis=0)

# how reservation works
Reservations are made by warehouses: we distribute the goods available in warehouses according to the orders closest to the warehouses
- We make a table with reserves - each record in the table will contain: order ID, warehouse ID, product ID, amount
- The df_inventories_nonreserved table stores the quantity of unreserved goods by warehouses
- For processing speed and the possibility of checking the correctness of the reserve, we save the record about the reserve in
   - df_orders_inventory ['reserved'] - increase the reserved amount
   - df_inventories_nonreserved - decrease the reserved amoun
    
For each warehouse:
  for each product with its stock:
  - we extract from the df_orders_inventory table the records with this product in which the product_count-reserved is greater than 0
  - join the resulting table with the order table using the order ID key - attach the column here - the distance to the warehouse
  - we sort the resulting table in ascending order of distance to the warehouse - that is, we get the nearest orders that the warehouse can serve
  - we take the top lines of the table and make reserves for them
    Thus, we got the lines of which we need to make reserves
    For each such line:
    - add an entry to the table with reserves
    - increase reserved in the original plate df_orders_inventory
    - we decrease the quantity of this product in this warehouse by df_inventories_nonreserved

In [ ]:
# reservation procedure
# table for account non reserved inventory in warehouses
df_inventories_nonreserved = df_inventories.copy()
df_orders_inventory['reserved'] = 0
# main reserve table - for a start it is empty
df_reserve = pd.DataFrame(columns = ["order_id","warehouse_id","product_id","amount","distance"],dtype="int32")
warehouse_id = 0
for warehouse_id in range(warehouses_count):
    # get all available goods on the warehouse
    wh_inventories = df_inventories[warehouse_id][df_inventories[warehouse_id] > 0]
    # iterate over each available product on warehouse
    for prod_id in wh_inventories.index:
        prod_amount = wh_inventories[prod_id]
        # find orders line records need this product and not reserved yet
        # we kno the only line for one product can be in this table
        orders = df_orders_inventory[(df_orders_inventory['product_id'] == prod_id) & \
                ((df_orders_inventory['product_count']- df_orders_inventory['reserved'] ) > 0)]
        # join to our dataframe distance info - from this warehouse to the order
        dist_orders = pd.merge(orders,df_orders,left_on='order_id',right_index=True,how='left') \
           [['order_id','product_id','product_count','reserved','wh_' + str(warehouse_id)]]
        # order invenory records by distance - more closer first
        ordered_dist_orders = dist_orders.sort_values(by=['wh_' + str(warehouse_id)], ascending=True)
        # iterate over inventory records from closer to farther and make reservation for them
        for order in ordered_dist_orders.iterrows():
            need_for_reserve = int(order[1]['product_count'] - order[1]['reserved'])
            order_order_id = order[1]['order_id']
            # calculate amount for reserve this product for the order
            available = df_inventories_nonreserved[warehouse_id][prod_id]
            if(available < need_for_reserve):
                # we can reserve available amount
                reserve_amount = available
            else:
                # we can reserve needed amount
                reserve_amount = need_for_reserve
            # add a record to reserve table    
            df_reserve = df_reserve.append({'order_id':order_order_id, \
                'warehouse_id':warehouse_id, \
                'product_id':prod_id, \
                'amount':reserve_amount, \
                'distance':order[1]['wh_'+str(warehouse_id)]}, ignore_index=True)

            # note reserve in df_orders_inventory
            # first find order line using product_id and order_id
            inventory_index = df_orders_inventory[(df_orders_inventory['order_id'] == order_order_id) & \
                (df_orders_inventory['product_id'] == prod_id)].index[0]
            
            df_orders_inventory.at[inventory_index,'reserved'] += reserve_amount
            # decreace available warehouse amount df_inventories_nonreserved
            df_inventories_nonreserved[warehouse_id][prod_id] -= reserve_amount
            # if product is finishes get to the next one
            if df_inventories_nonreserved[warehouse_id][prod_id] == 0:
                break

In [ ]:
# Check reservation was correct
total_reserved_1 = (df_inventories.sum() - df_inventories_nonreserved.sum()).sum()
total_reserved_2 = df_reserve['amount'].sum()
total_reserved_3 = df_orders_inventory['reserved'].sum()
print(total_reserved_1,total_reserved_2,total_reserved_3)
if(total_reserved_1 == total_reserved_2 == total_reserved_3):
    print("First check looks good")
else:
    print("First check looks wrong")

reserve_diff = df_orders_inventory[df_orders_inventory['product_count'] != df_orders_inventory['reserved']]
if(len(reserve_diff) == 0):
    print("Second check looks good")
else:
    print("Second check looks wrong")
    print(reserve_diff)


In [ ]:
# We estimate the total distance of all reserves from the warehouse where they are made
for wh_id in range(warehouses_count):
    reserves = df_reserve[df_reserve['warehouse_id'] == wh_id]
    distance = reserves['distance'].sum()
    print("For warehouse %s distance is %s " % (wh_id, distance))
    

In [ ]:
# TODO - visualize by reserves - for which warehouses which reserves are assigned 
# to visually assess the optimality

# Thoughts on route planning
- most of the orders - exceed the drone's carrying capacity - this means there is no point in complex algorithms for constructing a route graph - there will not be many points in any route
- it is necessary first of all to take away small orders and then larger ones
- it is necessary first of all to take away the closest orders and then the most distant ones
- the denser we load goods into the drone, the faster we will deliver everything - drones should leave the warehouses with the maximum load

## algorithm
- we choose the nearest unplanned reserve, and we plan an order for the route
- if there is room left in the drone:
     - select the 10 closest reserves to the current one and look for one that fits in the remaining space in the drone
     - planning it
     - repeat until there is room in the drone

## implementation of the routes formation
- initial drone capacity in the route sheet = 1
- filter by product_item_weight and available load capacity unplanned reserves - if the list is empty - close the route
- if the reserve is the first in the route:
  - we take the goods closest to the warehouse and plan it (we include as many units in the route as it will fit)
- if the reserve is not the first:
    - calculate the distance to everything else from the previous reserve
    - we discard everything that exceeds 1/10 of the map in distance - if as a result the list is empty - we close the route. there is no point in making big flights
    - we sort by distance to the last reserve what we have selected.
    - select the product closest to the last reserve and add it to the route.



In [ ]:
# attach the row and count columns from df_orders to the reserves so that we can quickly calculate the distance
df_reserve = pd.merge(df_reserve,df_orders[['row','column']],left_on='order_id',right_index=True,how='left')
# for a start planned reserves is 0 0
df_reserve['planned'] = 0
# Start from route number 0. After we finish to plan route increment route_index
route_index = 0



In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Join item_weight column to df_reserve based on product_id and product weights table
arr_prod_ids = np.array(df_reserve['product_id'].values,dtype="int")
df_reserve['product_item_weight'] = arr_products_weights[arr_prod_ids]/float(max_payload)

# table route/reserve connection - a lot of things - so that everything that is needed when we program the drones
df_reserve_route = pd.DataFrame(columns=["reserve_id","warehouse_id","route_id","order_id","row","column","product_id","amount","item_weight"])

for wh_id in range(warehouses_count):
    df_wh_reserves = df_reserve[df_reserve['warehouse_id'] == wh_id].sort_values('distance')
    # Loop - while there are unplanned reserves for this warehouse
    while(df_wh_reserves['amount'].sum() > df_wh_reserves['planned'].sum()):
        # planning one route
        # initial carrying capacity of the route
        route_workload = 1

        # Flag: remaining orders are not far
        orders_not_far = True
        reserve_no = 0
        # loop - until we have goods that fit into the route AND the remaining orders are close enough
        while(len(df_wh_reserves[df_wh_reserves['amount'] > df_wh_reserves['planned']]
                   [df_wh_reserves['product_item_weight'] <= route_workload] > 0)):
            # unplanned reserves that remain
            df_res_not_planned = df_wh_reserves[df_wh_reserves['amount'] > df_wh_reserves['planned']]\
                [df_wh_reserves['product_item_weight'] <= route_workload]

            if(not reserve_no): # first order in the route
                from_row,from_col = df_warehouses_locations.loc[wh_id]

            # determine the closest reserve
            df_res_not_planned['distance'] = ((from_row-df_res_not_planned['row'])**2 
                                            + (from_col-df_res_not_planned['column'])**2)**0.5
            nearest_reserve = df_res_not_planned.sort_values('distance').iloc[0]
            # if the distance to the nearest reserve is greater than the specified limit, 
            # we finish planning
            # TODO 40 it the constant by which we decide that the remaining orders are far away, 
            # can be optimized
            if(nearest_reserve.distance > 40 and reserve_no != 0):
                orders_not_far = False
                reserve_no += 1
                break
            # the number of units of goods from the reserve that will fit into the drone
            space_available = route_workload // nearest_reserve.product_item_weight
            # how much will we load
            amount_for_load = min([space_available,nearest_reserve.amount - nearest_reserve.planned])
            route_link = {
                "reserve_id": nearest_reserve.name,
                "warehouse_id": wh_id,
                "route_id" : route_index,
                "order_id" : nearest_reserve.order_id,
                "row" : nearest_reserve.row,
                "column" : nearest_reserve.column,
                "product_id" : nearest_reserve.product_id,
                "amount" : amount_for_load,
                "item_weight" : nearest_reserve.product_item_weight
            }
            route_workload -= amount_for_load*nearest_reserve.product_item_weight
            # Writing to the link table
            df_reserve_route = df_reserve_route.append(route_link,ignore_index = True)
            # in the table of reserves, we increase the planned by the reserve by amount_for_load
            df_wh_reserves.at[nearest_reserve.name,"planned"] = df_wh_reserves.at[nearest_reserve.name,"planned"] + amount_for_load
            df_reserve.at[nearest_reserve.name,"planned"] = df_wh_reserves.at[nearest_reserve.name,"planned"]
            from_row,from_col = nearest_reserve[['row','column']]
            reserve_no += 1
        route_index += 1
    

In [ ]:
# Let's check that we have compiled the routes correctly
for wh_id in range(warehouses_count):
    print("Checks for warehouse %s " % wh_id)
    if (df_reserve[df_reserve["warehouse_id"] == wh_id]['planned'].sum() == 
        df_reserve[df_reserve["warehouse_id"] == wh_id]['amount'].sum()):
        print("First check well")
    else:
        print("First check fails planned sum = %s amound = %s"%
              (df_reserve[df_reserve["warehouse_id"] == wh_id]['planned'].sum(),
              df_reserve[df_reserve["warehouse_id"] == wh_id]['amount'].sum()))
    if(df_reserve_route[df_reserve_route['warehouse_id'] == wh_id]['amount'].sum() == 
           df_reserve[df_reserve["warehouse_id"] == wh_id]['planned'].sum()):
        print("Second check well")
    else:
        print("Second check fails. Reserve route has %s items, but df_reserve has %s"%
             (df_reserve_route[df_reserve_route['warehouse_id'] == wh_id]['amount'].sum(),
             df_reserve[df_reserve["warehouse_id"] == wh_id]['planned'].sum()))
        
    if(set(df_reserve_route[df_reserve_route["warehouse_id"] == wh_id]['order_id']) == set(df_reserve[df_reserve['warehouse_id'] == wh_id]['order_id'])):
        print("Third check well")
    else:
        print("Third check fails df_reserve router orders are not equal to df_reserve orders")


# Lay out routes by drones
Calculate the total distance for each route
Create a table with routes where the index will be the set from df_reserve_route and the columns will be:
- total weight in the route (to check and assess the quality of planning - should be less than 1)
- total mileage between all points of the route (including return to warehouse + steps of loading and unloading)
- warehouse number
- drone number

Weigh the resulting routes by warehouses - that is, how many kilometers of the route are there for each warehouse

Distribute existing drones proportionally across warehouses

Estimate how much we can deliver from the total volume of orders

Distribute routes to drones

In [ ]:
# Preliminary preparation
route_ids = set(df_reserve_route['route_id'])
df_routes = pd.DataFrame(index = route_ids, columns=['weight','distance','warehouse_id','drone_id'])
# estimate the total weight in routes
df_reserve_route['reserve_weight'] = df_reserve_route['amount'] * df_reserve_route['item_weight']
# we group reserves / routes by routes, adding up the total weight
reserve_weight = df_reserve_route[['route_id','reserve_weight']].groupby('route_id').sum()
# push the resulting weight into df_routes
df_routes['weight'] = reserve_weight
# estimate of the drone load obtained during the reservation: how much drones are filled in routes
reserve_weight.hist(bins = 100)


In [ ]:
#visual assessment of what routes we got (reserves distributed along routes)
df_reserve_route.sort_values('route_id').head(20)


In [ ]:
# count the total number of turns for each route in order to evenly distribute the routes among the drones
for route in df_routes.index:
    # take route reserves
    r_reserve = df_reserve_route[df_reserve_route['route_id'] == route]
    # warehouse coordinates - from here we fly out and here we return
    wh_id = r_reserve['warehouse_id'].iloc[0]
    wh_row,wh_col = df_warehouses_locations.loc[wh_id]
    # get the coordinates of the reserve and group them by orders
    df_orders_locations = r_reserve[["order_id","row","column"]].groupby('order_id').mean()
    # calculate how many turns the route will take
    arr_points = np.zeros((df_orders_locations.shape[0]+3,4))
    arr_points[1:-2,:2] = np.array(df_orders_locations)
    arr_points[[0,-2],:2] = wh_row,wh_col
    arr_points[1:,2:] = arr_points[:-1,:2]
    mat = arr_points[1:-1,:]
    full_dist = np.sum(np.ceil(((mat[:,0]-mat[:,2])**2 + (mat[:,1] - mat[:,3])**2)**0.5))
    # add turns to load / unload
    # TODO did not take into account here that one turn is needed for unloading and 
    # loading for each product - not critical, but it can be done
    full_dist += full_dist + df_orders_locations.shape[0]+1
    # save what we counted
    df_routes.at[route,['distance','warehouse_id']] = full_dist,wh_id


In [ ]:
# turns by warehouses
df_wh_turns = df_routes[['warehouse_id','distance']].groupby('warehouse_id').sum()
# Do we manage to distribute all orders with existing drones 
# for the number of turns given to us?
df_wh_turns.sum().distance / (drones*turns)

# Divide drones between warehouses and distribute routes among drones
- we need all drones to deliver the fastest orders first and then the slowest ones from all warehouses
- it is necessary that all drones are loaded evenly and all work until the end of the simulation
- when the drone has finished delivering goods in its warehouse - it can help deliver goods from someone else's

## implementation
- we count the number of drones associated with warehouses in proportion to the mileage of routes from the same warehouses
- we make a table with drones - where we do the initial assignment of drones to warehouses


- we carry out planning for each warehouse and associated drones
- we plan to simultaneously scatter orders between drones - in such a way that all drones would first carry quick orders, then slowes ones.

In [ ]:
# count the number of drones associated with warehouses in proportion to the total distance of routes of the same warehouses
df_wh_turns['share'] = df_wh_turns['distance']/df_wh_turns.distance.sum()
df_wh_turns['drones'] = np.floor(df_wh_turns['share']*drones)
rest_drone_amount = int(drones - df_wh_turns['drones'].sum())
rest_indexes = df_wh_turns.sort_values('distance',ascending=False).iloc[:rest_drone_amount].index.values
rest_indexes
df_wh_turns.at[rest_indexes,'drones'] = df_wh_turns.loc[rest_indexes,'drones'] +1

# sign for assigning drones to warehouses
df_wh_drones = pd.DataFrame(index = range(drones),columns = ["wh_id"])
df_wh_drones['wh_id'] = [ wh_id for wh_id in range(warehouses_count) for c in range(int(df_wh_turns.loc[wh_id].drones)) ]

In [ ]:
# we carry out planning for each warehouse and associated drones
# we plan to simultaneously spread orders between drones in this way, 
# so that all drones first carry quick orders, then slow.
for wh_id in range(warehouses_count):
    # the total number of routes in the warehouse is divided by rounding up by the number of fixed drones
    route_per_drone = np.ceil(len(df_routes[df_routes.warehouse_id == wh_id] ) / df_wh_turns.loc[wh_id].drones)
    # we multiply the array with drone IDs for the given warehouse the resulting number of times and cut the result to the total number of routes in the warehouse
    # assign the resulting list of drone IDs to routes for this warehouse
    arr_drones = df_wh_drones[df_wh_drones.wh_id == wh_id].index.values
    arr2 = np.zeros((int(route_per_drone),len(arr_drones)))
    arr2[:,:] = arr_drones
    df_routes.loc[df_routes.warehouse_id == wh_id,['drone_id']] = \
        arr2.flatten()[:len(df_routes[df_routes.warehouse_id == wh_id] )]
    
    

In [ ]:
# check - do all routes have drones?
len(df_routes[df_routes.drone_id.isnull()])

In [ ]:
# we estimate turns by drones - to see who finishes earlier - who later
df_routes[['distance','drone_id']].groupby("drone_id").sum().plot()

In [ ]:
# Planned drone distances
drone_distances = df_routes[['distance','drone_id']].groupby("drone_id").sum().values
# distance spread estimation
np.std(drone_distances)/np.median(drone_distances)

# Drone Mutual Aid
The first distribution gives a high spread - that is, if everything is left like this, then some drones will finish earlier, while others will work additionally the same amount of time compared to those who finished

This is unfair and let those who finished earlier help those who did a lot of work.

## Algorithm

Grab the busiest drone and grab the least loaded drone based on the range of all connected routes. Subtract the shortest mileage from the longest

We take the tail of the routes from the most loaded drone (starting with the minimum mileage from the previous point) and divide it between 4 drones - three of which are the least loaded

We try to distribute the division of routes not by quantity, but by the same mileage for all 4 drones

We repeat the procedure until the estimate of the spread is set to 0.02

In [ ]:
while True:
    df_drone_distances = df_routes[['distance','drone_id']].groupby("drone_id").sum().sort_values('distance', ascending=False)
    most_loaded_drone = df_drone_distances.iloc[0]
    least_loaded_drone = df_drone_distances.iloc[-1]
    distance_diff = most_loaded_drone.values - least_loaded_drone.values

    # tail of the most loaded drone starting at least_loaded_drone.values
    df_routes_drone = df_routes[df_routes.drone_id == most_loaded_drone.name]
    df_routes_drone['dist_cum'] = df_routes_drone['distance'].cumsum()
    extra_routes = df_routes_drone[df_routes_drone.dist_cum > least_loaded_drone.values[0]]

    # the tail must be divided into 4 parts - and then - leave one alone and the other three removed and added
    # to the drones that are less loaded
    dist_ranges = np.linspace(extra_routes.iloc[0].dist_cum,extra_routes.iloc[-1].dist_cum,5)
    # for three pieces of tail
    for i in range(3):
        # routes of the next piece
        part_routes = df_routes_drone[(df_routes_drone.dist_cum > (dist_ranges[i+1])) & \
                                      (df_routes_drone.dist_cum < (dist_ranges[i+2]))].index
        # we take the least loaded drones for reloading
        reload_drone_id = df_drone_distances.iloc[-(i+1)].name
        # add routes to the selected drone. to do this, set drone_id to the id of this drone in df_routes
        df_routes.loc[part_routes,"drone_id"] = reload_drone_id
    # Planned drone distances
    drone_distances = df_routes[['distance','drone_id']].groupby("drone_id").sum().values
    # distance spread estimation
    deviation = np.std(drone_distances)/np.median(drone_distances)
    print("Deviation: %s Extra routes len: %s " % (deviation,len(extra_routes)))
    if(deviation < 0.02):
        break
    

In [ ]:
# check - all routes have drones
len(df_routes[df_routes.drone_id.isnull()]),len(df_routes)

In [ ]:
# we estimate the turnover by drones it is clear that in general the situation is better than it was before
df_routes[['distance','drone_id']].groupby("drone_id").sum().plot()

# Prepare submission


In [ ]:
# For each of the routes in df_routes, we form a set of lines - commands.
# Add the lines to the final list of lines
commands = []
for route_id in df_routes.index:
    p_route = df_reserve_route[df_reserve_route.route_id == route_id]
    drone_id = df_routes.loc[route_id].drone_id
    
    # Load command - for each position in the route
    for p_reserve in p_route.iterrows():
        command ="%d L %d %d %d" % (drone_id,
                                    p_reserve[1]['warehouse_id'],
                                    p_reserve[1]['product_id'],
                                    p_reserve[1]['amount'])
        commands.append(command)
    # Deliver commands - for each position in the route
    for p_reserve in p_route.iterrows():
        command = "%d D %d %d %d" % (drone_id,
                                    p_reserve[1]['order_id'],
                                    p_reserve[1]['product_id'],
                                    p_reserve[1]['amount'])
        commands.append(command)

In [ ]:
# We put the number of commands and the commands themselves for all drones in the submission.out file
total_commands = len(commands)
with open('./submission.csv','w') as f:
    f.write("%d\n" % total_commands)
    for command in commands:
        f.write("%s\n" % command)